In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sn
import warnings
from sklearn import preprocessing
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score
#from lightgbm import LGBMClassifier
import gc
import sys
warnings.filterwarnings('ignore')
plt.style.use('fivethirtyeight')
from sklearn.cluster import KMeans
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score
#pip install lightgbm --install-option=--gpu
import gc
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline

In [2]:
# Function to calculate missing values by column# Funct 
def missing_values_table(df):
        # Total missing values
        mis_val = df.isnull().sum()
                # Percentage of missing values
        mis_val_percent = 100 * df.isnull().sum() / len(df)     
        # Make a table with the results
        mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)        
        # Rename the columns
        mis_val_table_ren_columns = mis_val_table.rename(
        columns = {0 : 'Missing Values', 1 : '% of Total Values'})        
        # Sort the table by percentage of missing descending
        mis_val_table_ren_columns = mis_val_table_ren_columns[
            mis_val_table_ren_columns.iloc[:,1] != 0].sort_values(
        '% of Total Values', ascending=False).round(1)        
        # Print some summary information
        print ("Your selected dataframe has " + str(df.shape[1]) + " columns.\n"      
            "There are " + str(mis_val_table_ren_columns.shape[0]) +
              " columns that have missing values.")        
        # Return the dataframe with missing information
        return mis_val_table_ren_columns

In [3]:
#1- Cargo fichero bureau_balance.csv y veo cabecera 
BureauBalance = pd.read_csv('/Users/LENOVO/Downloads/bureau_balance.csv', delimiter= ',' , header=0)
#BureauBalance.head()
#BureauBalance
#BureauBalanceSNA.describe()
#BureauBalanceSNA = BureauBalance.dropna()
#BureauBalanceSNA


In [4]:
#BureauBalance.dtypes.value_counts()

In [5]:
#Función cuenta variables categoricas
def count_categoricas(df, group_var, df_name):
    """Calcula conteos y conteos normalizados para cada observación
    de `group_var` de cada categoría única en cada variable categórica
    
    Parámetros
    --------
    df : dataframe 
         El marco de datos para calcular el valor cuenta.
        
    group_var : string
        La variable por la cual agrupar el marco de datos. Para cada unico
        valor de esta variable, el marco de datos final tendrá una fila
        
    df_name : string
        Variable agregada al frente de los nombres de columna para realizar un seguimiento de las columnas

    
    Return
    --------
    categorical : dataframe
        Un marco de datos con recuentos y recuentos normalizados de cada categoría única en cada variable categórica
        con una fila por cada valor único de `group_var`
        
    """
    
    #Seleccion de las columnas categóricas
    categorica = pd.get_dummies(df.select_dtypes('object'))

    # Pone la de identificación en la columna
    categorica[group_var] = df[group_var]

    # Agrupa por el grupo var y calcula la suma y la media
    categorica = categorica.groupby(group_var).agg(['sum','mean'])
    
    column_names = []
    
    # Itera a través de las columnas en el nivel 0
    for var in categorica.columns.levels[0]:
        # Iterar a través de las estadísticas en el nivel 1
        for stat in ['count', 'count_norm']:
            # Genera un nuevo nombre de columna
            column_names.append('%s_%s_%s' % (df_name, var, stat))
    
    categorica.columns = column_names
    
    return categorica

In [6]:
#Función cuenta variables numéricas
def agg_numericas(df, group_var, df_name):
    """Agrega los valores numéricos en un marco de datos. Esto puede
    se utilizará para crear características para cada instancia de la variable de agrupación.
    
    Parameters
    --------
        df (dataframe): 
            el marco de datos para calcular las estadísticas
        group_var (string): 
            la variable por la cual agrupar df
        df_name (string):            
            la variable utilizada para renombrar las columnas
        
    Return
    --------
        agg (dataframe): 
            un marco de datos con las estadísticas agregadas para
            Todas las columnas numéricas. Cada instancia de la variable de agrupación tendrá
            las estadísticas (media, min, max, suma; actualmente admitidas) calculadas.
            Las columnas también se renombran para realizar un seguimiento de las características creadas.
    
    """
    # Elimina variables de identificación que no sean variables de agrupación
    for col in df:
        if col != group_var and 'SK_ID' in col:
            df = df.drop(columns = col)
            
    group_ids = df[group_var]
    numeric_df = df.select_dtypes('number')
    numeric_df[group_var] = group_ids

    # Agrupa por la variable especificada y calcula las estadísticas
    agg = numeric_df.groupby(group_var).agg(['count', 'mean', 'max', 'min', 'sum']).reset_index()

   #Crea nuevos nombres de columna
    columns = [group_var]

   #Itera a través de los nombres de las variables
    for var in agg.columns.levels[0]:
        # Salta la variable de agrupación
        if var != group_var:
            #Itera a través de los nombres de estadísticas
            for stat in agg.columns.levels[1][:-1]:
                # Hace un nuevo nombre de columna para la variable y estadística
                columns.append('%s_%s_%s' % (df_name, var, stat))

    agg.columns = columns
    return agg

In [7]:
#Recuento de cada tipo de estado de las variables CATEGORICAS para cada préstamo anterior por campo 'SK_ID_BUREAU'
BureauBalance_counts = count_categoricas(BureauBalance, group_var = 'SK_ID_BUREAU', df_name = 'bureau_balance')
#BureauBalance_counts.head()
#BureauBalance_counts.describe()
#BureauBalance_counts


In [8]:
# Calculo estadísticas de recuento de variables NUMERICAS para cada 'SK_ID_BUREAU' 
BureauBalance_agg = agg_numericas(BureauBalance, group_var = 'SK_ID_BUREAU', df_name = 'bureau_balance')
#BureauBalance_agg.head()
#BureauBalance_agg.describe()
#BureauBalance_agg


In [9]:
# Dataframe agrupando BureauBalance_agg con BureauBalance_counts por préstamo anterior
BureauBalance_by_loan = BureauBalance_agg.merge(BureauBalance_counts, right_index = True, left_on = 'SK_ID_BUREAU', how = 'outer')
#BureauBalance_by_loan.head()
#BureauBalance_by_loan.describe()
#BureauBalance_by_loan

In [10]:
#2- Cargo fichero bureau.csv
Bureau = pd.read_csv('/Users/LENOVO/Downloads/bureau.csv', delimiter= ',' , header=0)
#Bureau.head()
#Bureau.describe()
#BureauSNA = Bureau.dropna()
#BureauSNA

In [11]:
# Fusion para incluir la SK_ID_CURR
BureauBalance_by_loanMerge = BureauBalance_by_loan.merge(Bureau[['SK_ID_BUREAU', 'SK_ID_CURR']], on = 'SK_ID_BUREAU', how = 'left')
#BureauBalance_by_loanMerge
#BureauBalance_by_loanMerge.head()


In [12]:
##***** Agrega las estadísticas de prestamos por 'SK_ID_CURR'. Elimina columna 'SK_ID_BUREAU'. Para unir a train****
BureauBalance_by_client = agg_numericas(BureauBalance_by_loanMerge.drop(columns = ['SK_ID_BUREAU']), group_var = 'SK_ID_CURR', df_name = 'client')
#BureauBalance_by_client.head()
#BureauBalance_by_client.describe()
#BureauBalance_by_client

In [13]:
#BureauBalance_by_client.dtypes.value_counts()

In [14]:
# Missing values statistics
#missing_values = missing_values_table(BureauBalance_by_client)
#missing_values.head(20)

In [29]:
corrs = BureauBalance_by_client.corr()
#corrs

In [16]:
#corrs = corrs.sort_values('client_bureau_balance_MONTHS_BALANCE_count_mean', ascending = False)

# Ten most positive correlations
#pd.DataFrame(corrs['client_bureau_balance_MONTHS_BALANCE_count_mean'].head(10))

In [17]:
#2- Cargo fichero bureau.csv
#Bureau = pd.read_csv('/Users/LENOVO/Downloads/bureau.csv', delimiter= ',' , header=0)
#BureauSNA = Bureau.dropna()
#BureauSNA
#Bureau

In [18]:
#Se cuentan los prestamos anteriores por campo SK_ID_BUREAU y se agrupan por 'SK_ID_CURR'
Previous_loan_counts = Bureau.groupby('SK_ID_CURR', as_index=False)['SK_ID_BUREAU'].count().rename(columns = {'SK_ID_BUREAU': 'previous_loan_counts'})
#Previous_loan_counts

In [19]:
#Genera recuento y estadisticas de las variables para cada préstamo anterior eliminando columna 'SK_ID_BUREAU'y
#agrupando por campo 'SK_ID_CURR'
Bureau_agg = Bureau.drop(columns = ['SK_ID_BUREAU']).groupby('SK_ID_CURR',
as_index = False).agg(['count', 'mean', 'max', 'min', 'sum']).reset_index() 
#Bureau_agg.head()
#Bureau_agg

In [20]:
#Funcion para cambiar nombres en el df
# Lista de nombres de columna
columns = ['SK_ID_CURR']

#Itera a través de los nombres de las variables
for var in Bureau_agg.columns.levels[0]:
    # Salta el nombre de identificación
    if var != 'SK_ID_CURR':
        
        # Iterate through the stat names
        for stat in Bureau_agg.columns.levels[1][:-1]:
            # Itera a través de los nombres de estadísticas
            columns.append('bureau_%s_%s' % (var, stat))

In [21]:
#**** Asigna la lista de nombres de columnas como los nombres de columna del marco de datos***
Bureau_agg.columns = columns
#dfBureau_agg.head()
#Bureau_agg

In [22]:
categorical = pd.get_dummies(Bureau.select_dtypes('object'))
categorical['SK_ID_CURR'] = Bureau['SK_ID_CURR']
#categorical.head()
#categorical

In [23]:
#categorical_grouped.columns.levels[0][:10]

In [24]:
#categorical_grouped.columns.levels[1]

In [25]:
categorical_grouped = categorical.groupby('SK_ID_CURR').agg(['sum', 'mean'])
#categorical_grouped.head()

In [26]:
group_var = 'SK_ID_CURR'

# Need to create new column names
columns = []

# Iterate through the variables names
for var in categorical_grouped.columns.levels[0]:
    # Skip the grouping variable
    if var != group_var:
        # Iterate through the stat names
        for stat in ['count', 'count_norm']:
            # Make a new column name for the variable and stat
            columns.append('%s_%s' % (var, stat))

#  Rename the columns
categorical_grouped.columns = columns

categorical_grouped.head()

,CREDIT_ACTIVE_Active_count,CREDIT_ACTIVE_Active_count_norm,CREDIT_ACTIVE_Bad debt_count,CREDIT_ACTIVE_Bad debt_count_norm,CREDIT_ACTIVE_Closed_count,CREDIT_ACTIVE_Closed_count_norm,CREDIT_ACTIVE_Sold_count,CREDIT_ACTIVE_Sold_count_norm,CREDIT_CURRENCY_currency 1_count,CREDIT_CURRENCY_currency 1_count_norm,...,CREDIT_TYPE_Microloan_count,CREDIT_TYPE_Microloan_count_norm,CREDIT_TYPE_Mobile operator loan_count,CREDIT_TYPE_Mobile operator loan_count_norm,CREDIT_TYPE_Mortgage_count,CREDIT_TYPE_Mortgage_count_norm,CREDIT_TYPE_Real estate loan_count,CREDIT_TYPE_Real estate loan_count_norm,CREDIT_TYPE_Unknown type of loan_count,CREDIT_TYPE_Unknown type of loan_count_norm
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,
100001,3,0.428571,0,0.0,4,0.571429,0,0.0,7,1.0,...,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0
100002,2,0.250000,0,0.0,6,0.750000,0,0.0,8,1.0,...,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0
100003,1,0.250000,0,0.0,3,0.750000,0,0.0,4,1.0,...,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0
100004,0,0.000000,0,0.0,2,1.000000,0,0.0,2,1.0,...,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0
100005,2,0.666667,0,0.0,1,0.333333,0,0.0,3,1.0,...,0,0.0,0,0.0,0,0.0,0,0.0,0,0.0


In [27]:
# Dataframe agrupando Bureau_agg con categorical_grouped por préstamo anterior
BureauAgregado_by_loan = categorical_grouped.merge(Bureau_agg, right_index = True, left_on = 'SK_ID_CURR', how = 'outer')
#BureauAgregado_by_loan.head()
#BureauAgregado_by_loan.describe()
#BureauAgregado_by_loan

In [28]:
# Missing values statistics
#missing_values = missing_values_table(BureauAgregado_by_loan)
#missing_values.head(20)

In [30]:
corrs = BureauAgregado_by_loan.corr()
#corrs

In [31]:
#corrs = corrs.sort_values('CREDIT_ACTIVE_Closed_count_norm', ascending = False)

# Ten most positive correlations
#pd.DataFrame(corrs['CREDIT_ACTIVE_Closed_count_norm'].head(10))

In [32]:
def agg_numeric(df, parent_var, df_name):
    """
    Agrupa y agrega los valores numéricos en un marco de datos hijo
    por la variable padre.
    
    Parameters
    --------
        df (dataframe): 
            el marco de datos hijo para calcular las estadísticas en
        parent_var (string): 
            La variable principal utilizada para agrupar y agregar
        df_name (string): 
            la variable utilizada para renombrar las columnas
        
    Return
    --------
        agg (dataframe): 
            un marco de datos con las estadísticas agregadas por el `parent_var` para
            Todas las columnas numéricas. Cada observación de la variable principal tendrá
            una fila en el marco de datos con la variable principal como índice.
            Las columnas también se renombran usando el `df_name`. Columnas con todos los duplicados.
            Se eliminan los valores.
    
    """
      
    # Elimina variables de identificación que no sean variables de agrupación
    for col in df:
        if col != parent_var and 'SK_ID' in col:
            df = df.drop(columns = col)
            
    # Solo toma las variables numéricas
    parent_ids = df[parent_var].copy()
    numeric_df = df.select_dtypes('number').copy()
    numeric_df[parent_var] = parent_ids
    
    # Agrupa por la variable especificada y calcula las estadísticas
    agg = numeric_df.groupby(parent_var).agg(['count', 'mean', 'max', 'min', 'sum'])

    # Necesita crear nuevos nombres de columna
    columns = []

    # Itera a través de los nombres de las variables
    for var in agg.columns.levels[0]:
        if var != parent_var:
            # Itera a través de los nombres de estadísticas
            for stat in agg.columns.levels[1]:
                # Genera un nuevo nombre de columna para la variable y estadística
                columns.append('%s_%s_%s' % (df_name, var, stat))
    
    agg.columns = columns
    
    # Elimina las columnas con todos los valores redundantes
    _, idx = np.unique(agg, axis = 1, return_index=True)
    agg = agg.iloc[:, idx]
    
    return agg

In [33]:
def agg_categorical(df, parent_var, df_name):
    """
    Agrega las características categóricas en un marco de datos hijo
    para cada observación de la variable principal.
    
    Parameters
    --------
    df : dataframe 
        El marco de datos para calcular el valor cuenta.
        
    parent_var : string
        La variable por la cual agrupar y agregar el marco de datos. Para cada unico
        valor de esta variable, el marco de datos final tendrá una fila
        
    df_name : string
       Variable agregada al frente de los nombres de columna para realizar un seguimiento de las columnas

    
    Return
    --------
    categorical : dataframe
        Un marco de datos con estadísticas agregadas para cada observación de parent_var
        Las columnas también se renombran y las columnas con valores duplicados se eliminan.
        
    """
       
    # Selecciona las columnas categóricas
    categorical = pd.get_dummies(df.select_dtypes('category'))

    # Asegura poner la identificación de identificación en la columna
    categorical[parent_var] = df[parent_var]

    # Agrupa por el grupo var y calcula la suma y la media
    categorical = categorical.groupby(parent_var).agg(['sum', 'count', 'mean'])
    
    column_names = []
    
    # Iterar a través de las columnas en el nivel 0
    for var in categorical.columns.levels[0]:
        # Iterate through the stats in level 1
        for stat in ['sum', 'count', 'mean']:
            # Iterar a través de las estadísticas en el nivel 1
            column_names.append('%s_%s_%s' % (df_name, var, stat))
    
    categorical.columns = column_names
        
    # Elimina columnas duplicadas por valores
    _, idx = np.unique(categorical, axis = 1, return_index = True)
    categorical = categorical.iloc[:, idx]
    
    return categorical

In [34]:
def aggregate_client(df, group_vars, df_names):
    """Agregar un marco de datos con datos a nivel de préstamo
    a nivel del cliente
    
    Args:
        df (dataframe): datos a nivel de préstamo
        group_vars (lista de dos cadenas): agrupando variables para el préstamo
        y luego el cliente (ejemplo ['SK_ID_PREV', 'SK_ID_CURR'])
        nombres (lista de dos cadenas): nombres para llamar a las columnas resultantes
        (ejemplo ['efectivo', 'cliente'])
        
    Returns:
        df_client (dataframe): estadísticas numéricas agregadas a nivel del cliente.
       Cada cliente tendrá una sola fila con todos los datos numéricos agregados
    """
    
    # Agrega las columnas numéricas
    df_agg = agg_numeric(df, parent_var = group_vars[0], df_name = df_names[0])
    
    # Si hay variables categóricas
    if any(df.dtypes == 'category'):
    
        # Cuenta las columnas categóricas
        df_counts = agg_categorical(df, parent_var = group_vars[0], df_name = df_names[0])

        # Fusiona la numérica y categórica
        df_by_loan = df_counts.merge(df_agg, on = group_vars[0], how = 'outer')

        gc.enable()
        del df_agg, df_counts
        gc.collect()

        # Combina para obtener la identificación del cliente en el marco de datos
        df_by_loan = df_by_loan.merge(df[[group_vars[0], group_vars[1]]], on = group_vars[0], how = 'left')

        # Elimina la identificación del préstamo
        df_by_loan = df_by_loan.drop(columns = [group_vars[0]])

        # Agrega estadísticas numéricas por columna
        df_by_client = agg_numeric(df_by_loan, parent_var = group_vars[1], df_name = df_names[1])

        
    # No hay variables categóricas
    else:
        # Combina para obtener la identificación del cliente en el marco de datos
        df_by_loan = df_agg.merge(df[[group_vars[0], group_vars[1]]], on = group_vars[0], how = 'left')
        
        gc.enable()
        del df_agg
        gc.collect()
        
        # Elimina la identificación del préstamo
        df_by_loan = df_by_loan.drop(columns = [group_vars[0]])
                
        # Agrega estadísticas numéricas por columna
        df_by_client = agg_numeric(df_by_loan, parent_var = group_vars[1], df_name = df_names[1])
        
    # Gestión de la memoria
    gc.enable()
    del df, df_by_loan
    gc.collect()

    return df_by_client

In [35]:
#3-Cargo fichero installments_payments.csv y veo cabecera
installments_payments = pd.read_csv('/Users/LENOVO/Downloads/installments_payments.csv', delimiter= ',' , header=0)
#installments_payments.head()
#installments_payments
#installments_paymentsSNA = installments_payments.dropna()
#installments_paymentsSNA

In [37]:
#installments_payments.dtypes.value_counts()

In [38]:
##***** Agrega las estadísticas de prestamos por 'SK_ID_CURR'. Elimina columna 'SK_ID_PREV'****
installments_payments_by_client = aggregate_client(installments_payments, group_vars = ['SK_ID_PREV', 'SK_ID_CURR'], df_names = ['installments', 'client'])
#installments_payments_by_client.head()
#installments_payments_by_client

In [39]:
# Missing values statistics
#missing_values = missing_values_table(installments_payments_by_client)
#missing_values.head(20)

In [40]:
corrs = installments_payments_by_client.corr()
#corrs

In [41]:
#corrs = corrs.sort_values('client_installments_DAYS_INSTALMENT_mean_sum', ascending = False)

# Ten most positive correlations
#pd.DataFrame(corrs['client_installments_DAYS_INSTALMENT_mean_sum'].head(10))

In [42]:
#4-Cargo fichero POS_CASH_balance.csv y veo cabecera
POS_CASH_balance = pd.read_csv('/Users/LENOVO/Downloads/POS_CASH_balance.csv', delimiter= ',' , header=0)
#POS_CASH_balance.head()
#POS_CASH_balance
#POS_CASH_balanceSNA = POS_CASH_balance.dropna()
#POS_CASH_balanceSNA

In [44]:
#POS_CASH_balance.dtypes.value_counts()

In [45]:
##***** Agrega las estadísticas de cash por 'SK_ID_CURR'. Elimina columna 'SK_ID_PREV'****
POS_CASH_balance_by_client = aggregate_client(POS_CASH_balance, group_vars = ['SK_ID_PREV', 'SK_ID_CURR'], df_names = ['cash', 'client'])
#POS_CASH_balance_by_client.head()
#POS_CASH_balance_by_client

In [46]:
# Missing values statistics
#missing_values = missing_values_table(POS_CASH_balance_by_client)
#missing_values.head(20)

In [47]:
corrs = POS_CASH_balance_by_client.corr()
#corrs

In [48]:
#corrs = corrs.sort_values('client_cash_CNT_INSTALMENT_min_sum', ascending = False)

# Ten most positive correlations
#pd.DataFrame(corrs['client_cash_CNT_INSTALMENT_min_sum'].head(10))

In [49]:
#5-Cargo fichero credit_card_balance.csv y veo cabecera
credit_card_balance = pd.read_csv('/Users/LENOVO/Downloads/credit_card_balance.csv', delimiter= ',' , header=0)
#credit_card_balance.head()
#credit_card_balance
#credit_card_balanceSNA = credit_card_balance.dropna()
#credit_card_balanceSNA

In [51]:
#credit_card_balance.dtypes.value_counts()

In [52]:
##***** Agrega las estadísticas de credit_card_balance por 'SK_ID_CURR'.Elimina columna 'SK_ID_PREV'
credit_card_balance_by_client = aggregate_client(credit_card_balance, group_vars = ['SK_ID_PREV', 'SK_ID_CURR'], df_names = ['credit', 'client'])
#credit_card_balance_by_client.head()
#credit_card_balance_by_client

In [54]:
# Missing values statistics
#missing_values = missing_values_table(credit_card_balance_by_client)
#missing_values.head(20)

In [55]:
corrs = credit_card_balance_by_client.corr()
#corrs

In [56]:
#corrs = corrs.sort_values('client_credit_MONTHS_BALANCE_min_sum', ascending = False)

# Ten most positive correlations
#pd.DataFrame(corrs['client_credit_MONTHS_BALANCE_min_sum'].head(10))

In [57]:
import sys

def return_size(df):
    """Return size of dataframe in gigabytes"""
    return round(sys.getsizeof(df) / 1e9, 2)

def convert_types(df, print_info = False):
    
    original_memory = df.memory_usage().sum()
    
    # Iterate through each column
    for c in df:
        
        # Convert ids and booleans to integers
        if ('SK_ID' in c):
            df[c] = df[c].fillna(0).astype(np.int32)
            
        # Convert objects to category
        elif (df[c].dtype == 'object') and (df[c].nunique() < df.shape[0]):
            df[c] = df[c].astype('category')
        
        # Booleans mapped to integers
        elif list(df[c].unique()) == [1, 0]:
            df[c] = df[c].astype(bool)
        
        # Float64 to float32
        elif df[c].dtype == float:
            df[c] = df[c].astype(np.float32)
            
        # Int64 to int32
        elif df[c].dtype == int:
            df[c] = df[c].astype(np.int32)
        
    new_memory = df.memory_usage().sum()
    
    if print_info:
        print(f'Original Memory Usage: {round(original_memory / 1e9, 2)} gb.')
        print(f'New Memory Usage: {round(new_memory / 1e9, 2)} gb.')
        
    return df

In [58]:
#6-Cargo fichero previous_application.csv y veo cabecera
#previous_application = pd.read_csv('/Users/LENOVO/Downloads/previous_application.csv', delimiter= ',' , header=0)
#previous_application.head()
#previous_application
#previous_applicationSNA = previous_application.dropna()
#previous_applicationSNA
previous = pd.read_csv('/Users/LENOVO/Downloads/previous_application.csv', delimiter= ',' , header=0)
previous = convert_types(previous, print_info=True)
#previous.head()

Original Memory Usage: 0.49 gb.
New Memory Usage: 0.18 gb.


In [59]:
#previous_application.dtypes.value_counts()
#previous_applicationSNA.dtypes.value_counts()
#previous.dtypes.value_counts()

In [60]:
# Calculate aggregate statistics for each numeric column
#previous_application_agg = agg_numeric(previous_application, 'SK_ID_CURR', 'previous')
#previous_application_agg.head()
#previous_application_agg
previous_agg = agg_numeric(previous, 'SK_ID_CURR', 'previous')
#previous_agg

In [61]:
# Calculate value counts for each categorical column
previous_counts = agg_categorical(previous, 'SK_ID_CURR', 'previous')
print('Previous counts shape: ', previous_counts.shape)
#previous_counts.head()

Previous counts shape:  (338857, 285)


In [62]:
# Dataframe agrupando BureauBalance_agg con BureauBalance_counts por préstamo anterior
previous_application_ag = previous_agg.merge(previous_counts, right_index = True, left_on = 'SK_ID_CURR', how = 'outer')
#previous_application_ag

In [63]:
# Missing values statistics
#missing_values = missing_values_table(previous_application_ag)
#missing_values.head(20)

In [64]:
corrs = previous_application_ag.corr()
#corrs

In [65]:
#corrs = corrs.sort_values('previous_DAYS_FIRST_DUE_max', ascending = False)

# Ten most positive correlations
#pd.DataFrame(corrs['previous_DAYS_FIRST_DUE_max'].head(10))

In [66]:
#7-Cargo fichero application_train.csv y veo cabecera
train = pd.read_csv('/Users/LENOVO/Downloads/application_train.csv', delimiter= ',' , header=0)
#train

In [67]:
#train['TARGET'].astype(int).plot.hist();

In [68]:
# Missing values statistics
#missing_values = missing_values_table(train)
#missing_values.head(20)

In [69]:
#train.dtypes.value_counts()

In [65]:
# Number of unique classes in each object column
#train.select_dtypes('object').apply(pd.Series.nunique, axis = 0)

In [70]:
# Find correlations with the target and sort
correlations = train.corr()['TARGET'].sort_values()

In [71]:
# Display correlations
print('Most Positive Correlations:\n', correlations.tail(15))
print('\nMost Negative Correlations:\n', correlations.head(15))

Most Positive Correlations:
 DEF_60_CNT_SOCIAL_CIRCLE       0.031276
DEF_30_CNT_SOCIAL_CIRCLE       0.032248
LIVE_CITY_NOT_WORK_CITY        0.032518
OWN_CAR_AGE                    0.037612
DAYS_REGISTRATION              0.041975
FLAG_DOCUMENT_3                0.044346
REG_CITY_NOT_LIVE_CITY         0.044395
FLAG_EMP_PHONE                 0.045982
REG_CITY_NOT_WORK_CITY         0.050994
DAYS_ID_PUBLISH                0.051457
DAYS_LAST_PHONE_CHANGE         0.055218
REGION_RATING_CLIENT           0.058899
REGION_RATING_CLIENT_W_CITY    0.060893
DAYS_BIRTH                     0.078239
TARGET                         1.000000
Name: TARGET, dtype: float64

Most Negative Correlations:
 EXT_SOURCE_3                 -0.178919
EXT_SOURCE_2                 -0.160472
EXT_SOURCE_1                 -0.155317
DAYS_EMPLOYED                -0.044932
FLOORSMAX_AVG                -0.044003
FLOORSMAX_MEDI               -0.043768
FLOORSMAX_MODE               -0.043226
AMT_GOODS_PRICE              -0.039645


In [72]:
# Join to the training dataframe BureauBalance_by_client
train = train.merge(BureauBalance_by_client, on = 'SK_ID_CURR', how = 'left')

# Fill the missing values with 0 
#train['SK_ID_CURR'] = train['SK_ID_CURR'].fillna('mean')
#train.head()
#train

In [73]:
#missing_values = missing_values_table(train)
#missing_values.head(20)

In [75]:
#train.dtypes.value_counts()

In [76]:
correlations = train.corr()['TARGET'].sort_values()
correlations

EXT_SOURCE_3                                      -0.178919
EXT_SOURCE_2                                      -0.160472
EXT_SOURCE_1                                      -0.155317
client_bureau_balance_MONTHS_BALANCE_count_mean   -0.080193
client_bureau_balance_MONTHS_BALANCE_count_max    -0.068792
                                                     ...   
client_bureau_balance_MONTHS_BALANCE_min_min       0.073225
client_bureau_balance_MONTHS_BALANCE_mean_mean     0.076424
DAYS_BIRTH                                         0.078239
client_bureau_balance_MONTHS_BALANCE_min_mean      0.089038
TARGET                                             1.000000
Name: TARGET, Length: 211, dtype: float64

In [77]:
# Join to the training dataframe BureauAgregado_by_loan
train = train.merge(BureauAgregado_by_loan, on = 'SK_ID_CURR', how = 'left')
# Fill the missing values with 0 
#train['SK_ID_CURR'] = train['SK_ID_CURR'].fillna('mean')
#train

In [79]:
#train.dtypes.value_counts()

In [80]:
#missing_values = missing_values_table(train)
#missing_values.head(20)

In [81]:
correlations = train.corr()['TARGET'].sort_values()
correlations

EXT_SOURCE_3                                      -0.179433
EXT_SOURCE_2                                      -0.159119
EXT_SOURCE_1                                      -0.154810
client_bureau_balance_MONTHS_BALANCE_count_mean   -0.081162
CREDIT_ACTIVE_Closed_count_norm                   -0.079369
                                                     ...   
client_bureau_balance_MONTHS_BALANCE_mean_mean     0.076891
CREDIT_ACTIVE_Active_count_norm                    0.077356
DAYS_BIRTH                                         0.078222
client_bureau_balance_MONTHS_BALANCE_min_mean      0.089774
TARGET                                             1.000000
Name: TARGET, Length: 317, dtype: float64

In [82]:
# Join to the training dataframe installments_payments_by_client
train = train.merge(installments_payments_by_client, on = 'SK_ID_CURR', how = 'left')
# Fill the missing values with 0 
#train['SK_ID_CURR'] = train['SK_ID_CURR'].fillna('mean')
#train

In [84]:
#train.dtypes.value_counts()

In [85]:
#missing_values = missing_values_table(train)
#missing_values.head(20)

In [86]:
correlations = train.corr()['TARGET'].sort_values()
correlations

EXT_SOURCE_3                                      -0.179433
EXT_SOURCE_2                                      -0.159119
EXT_SOURCE_1                                      -0.154810
client_bureau_balance_MONTHS_BALANCE_count_mean   -0.081162
CREDIT_ACTIVE_Closed_count_norm                   -0.079369
                                                     ...   
client_bureau_balance_MONTHS_BALANCE_mean_mean     0.076891
CREDIT_ACTIVE_Active_count_norm                    0.077356
DAYS_BIRTH                                         0.078222
client_bureau_balance_MONTHS_BALANCE_min_mean      0.089774
TARGET                                             1.000000
Name: TARGET, Length: 423, dtype: float64

In [87]:
# Join to the training dataframe POS_CASH_balance_by_client
train = train.merge(POS_CASH_balance_by_client, on = 'SK_ID_CURR', how = 'left')
# Fill the missing values with 0 
#train['SK_ID_CURR'] = train['SK_ID_CURR'].fillna('mean')
#train

In [90]:
#train.dtypes.value_counts()

In [91]:
#missing_values = missing_values_table(train)
#missing_values.head(20)

In [92]:
correlations = train.corr()['TARGET'].sort_values()
correlations

EXT_SOURCE_3                                      -0.179433
EXT_SOURCE_2                                      -0.159119
EXT_SOURCE_1                                      -0.154810
client_bureau_balance_MONTHS_BALANCE_count_mean   -0.081162
CREDIT_ACTIVE_Closed_count_norm                   -0.079369
                                                     ...   
client_bureau_balance_MONTHS_BALANCE_mean_mean     0.076891
CREDIT_ACTIVE_Active_count_norm                    0.077356
DAYS_BIRTH                                         0.078222
client_bureau_balance_MONTHS_BALANCE_min_mean      0.089774
TARGET                                             1.000000
Name: TARGET, Length: 517, dtype: float64

In [93]:
# Join to the training dataframe credit_card_balance_by_client
train = train.merge(credit_card_balance_by_client, on = 'SK_ID_CURR', how = 'left')
# Fill the missing values with 0 
#train['SK_ID_CURR'] = train['SK_ID_CURR'].fillna('mean')
#train

In [95]:
#train.dtypes.value_counts()

In [96]:
#missing_values = missing_values_table(train)
#missing_values.head(20)

In [97]:
correlations = train.corr()['TARGET'].sort_values()
correlations

EXT_SOURCE_3                                       -0.179433
EXT_SOURCE_2                                       -0.159119
EXT_SOURCE_1                                       -0.154810
client_bureau_balance_MONTHS_BALANCE_count_mean    -0.081162
CREDIT_ACTIVE_Closed_count_norm                    -0.079369
                                                      ...   
client_credit_CNT_DRAWINGS_ATM_CURRENT_mean_mean    0.106478
client_credit_CNT_DRAWINGS_ATM_CURRENT_mean_max     0.107244
TARGET                                              1.000000
client_credit_SK_DPD_min_mean                            NaN
client_credit_SK_DPD_min_sum                             NaN
Name: TARGET, Length: 848, dtype: float64

In [98]:
# Join to the training dataframe previous_application_ag
train = train.merge(previous_application_ag, on = 'SK_ID_CURR', how = 'left')
# Fill the missing values with 0 
#train['SK_ID_CURR'] = train['SK_ID_CURR'].fillna('mean')
#train

In [100]:
#train.dtypes.value_counts()

In [101]:
#missing_values = missing_values_table(train)
#missing_values.head(20)

In [102]:
correlations = train.corr()['TARGET'].sort_values()
correlations

EXT_SOURCE_3                                           -0.179433
EXT_SOURCE_2                                           -0.159119
EXT_SOURCE_1                                           -0.154810
client_bureau_balance_MONTHS_BALANCE_count_mean        -0.081162
CREDIT_ACTIVE_Closed_count_norm                        -0.079369
                                                          ...   
TARGET                                                  1.000000
client_credit_SK_DPD_min_mean                                NaN
client_credit_SK_DPD_min_sum                                 NaN
previous_NAME_GOODS_CATEGORY_House Construction_mean         NaN
previous_NAME_GOODS_CATEGORY_House Construction_sum          NaN
Name: TARGET, Length: 1213, dtype: float64

In [103]:
# Display correlations
print('Most Positive Correlations:\n', correlations.tail(15))
print('\nMost Negative Correlations:\n', correlations.head(15))

Most Positive Correlations:
 client_credit_AMT_TOTAL_RECEIVABLE_mean_max             0.086666
client_credit_AMT_BALANCE_mean_mean                     0.086739
client_credit_AMT_BALANCE_mean_max                      0.087329
client_bureau_balance_MONTHS_BALANCE_min_mean           0.089774
client_credit_CNT_DRAWINGS_CURRENT_max_min              0.099495
client_credit_CNT_DRAWINGS_CURRENT_max_mean             0.099831
client_credit_CNT_DRAWINGS_CURRENT_max_max              0.100397
client_credit_CNT_DRAWINGS_ATM_CURRENT_mean_min         0.106376
client_credit_CNT_DRAWINGS_ATM_CURRENT_mean_mean        0.106478
client_credit_CNT_DRAWINGS_ATM_CURRENT_mean_max         0.107244
TARGET                                                  1.000000
client_credit_SK_DPD_min_mean                                NaN
client_credit_SK_DPD_min_sum                                 NaN
previous_NAME_GOODS_CATEGORY_House Construction_mean         NaN
previous_NAME_GOODS_CATEGORY_House Construction_sum          

In [104]:
train

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,previous_CODE_REJECT_REASON_XAP_mean,previous_FLAG_LAST_APPL_PER_CONTRACT_Y_mean,previous_NAME_PORTFOLIO_POS_sum,previous_NAME_CONTRACT_TYPE_Consumer loans_sum,previous_NAME_CASH_LOAN_PURPOSE_XAP_sum,previous_NAME_PRODUCT_TYPE_XNA_sum,previous_NAME_CONTRACT_STATUS_Approved_sum,previous_CODE_REJECT_REASON_XAP_sum,previous_FLAG_LAST_APPL_PER_CONTRACT_Y_sum,previous_NAME_CONTRACT_TYPE_Cash loans_count
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,1.00,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
1,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,1.00,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
2,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,1.00,1.0,2.0,2.0,2.0,2.0,3.0,3.0,3.0,3.0
3,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,1.00,1.0,2.0,2.0,2.0,2.0,3.0,3.0,3.0,3.0
4,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,...,1.00,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
418516,456251,0,Cash loans,M,N,N,0,157500.0,254700.0,27558.0,...,1.00,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
418517,456252,0,Cash loans,F,N,Y,0,72000.0,269550.0,12001.5,...,1.00,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
418518,456253,0,Cash loans,F,N,Y,0,153000.0,677664.0,29979.0,...,1.00,1.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
418519,456254,1,Cash loans,F,N,Y,0,171000.0,370107.0,20205.0,...,1.00,1.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0


In [105]:
#8-Cargo fichero application_test.csv y veo cabecera
test = pd.read_csv('/Users/LENOVO/Downloads/application_test.csv', delimiter= ',' , header=0)
#test

In [108]:
#test.dtypes.value_counts()

In [109]:
#missing_values = missing_values_table(test)
#missing_values.head(20)

In [110]:
# Create a label encoder object
le = preprocessing.LabelEncoder()
le_count = 0

In [111]:
# Iterate through the columns
for col in train:
    if train[col].dtype == 'object':
        # If 2 or fewer unique categories
        if len(list(train[col].unique())) <= 2:
            # Train on the training data
            le.fit(train[col])
            # Transform both training and testing data
            train[col] = le.transform(train[col])
            test[col] = le.transform(test[col])
            
            # Keep track of how many columns were label encoded
            le_count += 1
            
print('%d columns were label encoded.' % le_count)

3 columns were label encoded.


In [112]:
# one-hot encoding of categorical variables
train = pd.get_dummies(train)
test  = pd.get_dummies(test )

print('Training Features shape: ', train.shape)
print('Testing Features shape: ', test.shape)

Training Features shape:  (418521, 1350)
Testing Features shape:  (48744, 239)


In [113]:
train_labels = train['TARGET']

# Align the training and testing data, keep only columns present in both dataframes
train, test = train.align(test, join = 'inner', axis = 1)

# Add the target back in
train['TARGET'] = train_labels

print('Training Features shape: ', train.shape)
print('Testing Features shape: ', test.shape)

Training Features shape:  (418521, 240)
Testing Features shape:  (48744, 239)


In [114]:
# Threshold for removing correlated variables
threshold = 0.9

# Absolute value correlation matrix
corr_matrix = train.corr().abs()
corr_matrix.head()

,SK_ID_CURR,NAME_CONTRACT_TYPE,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,...,WALLSMATERIAL_MODE_Block,WALLSMATERIAL_MODE_Mixed,WALLSMATERIAL_MODE_Monolithic,WALLSMATERIAL_MODE_Others,WALLSMATERIAL_MODE_Panel,"WALLSMATERIAL_MODE_Stone, brick",WALLSMATERIAL_MODE_Wooden,EMERGENCYSTATE_MODE_No,EMERGENCYSTATE_MODE_Yes,TARGET
SK_ID_CURR,1.000000,0.002222,0.001624,0.001251,0.002604,0.003304,0.003072,0.001117,0.002821,0.001519,...,0.001490,0.000289,0.000548,0.001327,0.001110,0.002119,0.000342,0.001310,0.002766,0.000160
NAME_CONTRACT_TYPE,0.002222,1.000000,0.003864,0.068262,0.030043,0.003473,0.223200,0.242975,0.187596,0.026582,...,0.007036,0.004244,0.011558,0.001557,0.011013,0.014085,0.000828,0.026938,0.000047,0.030325
FLAG_OWN_CAR,0.001624,0.003864,1.000000,0.002298,0.103301,0.071195,0.115429,0.141798,0.119637,0.041617,...,0.006010,0.000809,0.011491,0.003705,0.009519,0.025952,0.008881,0.034645,0.007992,0.021419
FLAG_OWN_REALTY,0.001251,0.068262,0.002298,1.000000,0.002530,0.002435,0.040455,0.005409,0.046751,0.014231,...,0.000351,0.003165,0.000489,0.001153,0.014763,0.017784,0.014839,0.021819,0.008897,0.005133
CNT_CHILDREN,0.002604,0.030043,0.103301,0.002530,1.000000,0.012044,0.002652,0.023044,0.001276,0.023980,...,0.004902,0.001168,0.001804,0.002183,0.019695,0.025326,0.011178,0.037425,0.004098,0.019374


In [115]:
# Upper triangle of correlations
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))
upper.head()

,SK_ID_CURR,NAME_CONTRACT_TYPE,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,...,WALLSMATERIAL_MODE_Block,WALLSMATERIAL_MODE_Mixed,WALLSMATERIAL_MODE_Monolithic,WALLSMATERIAL_MODE_Others,WALLSMATERIAL_MODE_Panel,"WALLSMATERIAL_MODE_Stone, brick",WALLSMATERIAL_MODE_Wooden,EMERGENCYSTATE_MODE_No,EMERGENCYSTATE_MODE_Yes,TARGET
SK_ID_CURR,NaN,0.002222,0.001624,0.001251,0.002604,0.003304,0.003072,0.001117,0.002821,0.001519,...,0.001490,0.000289,0.000548,0.001327,0.001110,0.002119,0.000342,0.001310,0.002766,0.000160
NAME_CONTRACT_TYPE,NaN,NaN,0.003864,0.068262,0.030043,0.003473,0.223200,0.242975,0.187596,0.026582,...,0.007036,0.004244,0.011558,0.001557,0.011013,0.014085,0.000828,0.026938,0.000047,0.030325
FLAG_OWN_CAR,NaN,NaN,NaN,0.002298,0.103301,0.071195,0.115429,0.141798,0.119637,0.041617,...,0.006010,0.000809,0.011491,0.003705,0.009519,0.025952,0.008881,0.034645,0.007992,0.021419
FLAG_OWN_REALTY,NaN,NaN,NaN,NaN,0.002530,0.002435,0.040455,0.005409,0.046751,0.014231,...,0.000351,0.003165,0.000489,0.001153,0.014763,0.017784,0.014839,0.021819,0.008897,0.005133
CNT_CHILDREN,NaN,NaN,NaN,NaN,NaN,0.012044,0.002652,0.023044,0.001276,0.023980,...,0.004902,0.001168,0.001804,0.002183,0.019695,0.025326,0.011178,0.037425,0.004098,0.019374


In [116]:
upper

,SK_ID_CURR,NAME_CONTRACT_TYPE,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,...,WALLSMATERIAL_MODE_Block,WALLSMATERIAL_MODE_Mixed,WALLSMATERIAL_MODE_Monolithic,WALLSMATERIAL_MODE_Others,WALLSMATERIAL_MODE_Panel,"WALLSMATERIAL_MODE_Stone, brick",WALLSMATERIAL_MODE_Wooden,EMERGENCYSTATE_MODE_No,EMERGENCYSTATE_MODE_Yes,TARGET
SK_ID_CURR,NaN,0.002222,0.001624,0.001251,0.002604,0.003304,0.003072,0.001117,0.002821,0.001519,...,0.001490,0.000289,0.000548,0.001327,0.001110,0.002119,0.000342,0.001310,0.002766,0.000160
NAME_CONTRACT_TYPE,NaN,NaN,0.003864,0.068262,0.030043,0.003473,0.223200,0.242975,0.187596,0.026582,...,0.007036,0.004244,0.011558,0.001557,0.011013,0.014085,0.000828,0.026938,0.000047,0.030325
FLAG_OWN_CAR,NaN,NaN,NaN,0.002298,0.103301,0.071195,0.115429,0.141798,0.119637,0.041617,...,0.006010,0.000809,0.011491,0.003705,0.009519,0.025952,0.008881,0.034645,0.007992,0.021419
FLAG_OWN_REALTY,NaN,NaN,NaN,NaN,0.002530,0.002435,0.040455,0.005409,0.046751,0.014231,...,0.000351,0.003165,0.000489,0.001153,0.014763,0.017784,0.014839,0.021819,0.008897,0.005133
CNT_CHILDREN,NaN,NaN,NaN,NaN,NaN,0.012044,0.002652,0.023044,0.001276,0.023980,...,0.004902,0.001168,0.001804,0.002183,0.019695,0.025326,0.011178,0.037425,0.004098,0.019374
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"WALLSMATERIAL_MODE_Stone, brick",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.068984,0.491403,0.008586,0.012739
WALLSMATERIAL_MODE_Wooden,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.077693,0.278281,0.006993
EMERGENCYSTATE_MODE_No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.090742,0.042851
EMERGENCYSTATE_MODE_Yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.004067


In [117]:
# Select columns with correlations above threshold
to_drop = [column for column in upper.columns if any(upper[column] > threshold)]

print('There are %d columns to remove.' % (len(to_drop)))

There are 39 columns to remove.


In [118]:
to_drop

['AMT_GOODS_PRICE',
 'FLAG_EMP_PHONE',
 'REGION_RATING_CLIENT_W_CITY',
 'LIVINGAPARTMENTS_AVG',
 'LIVINGAREA_AVG',
 'APARTMENTS_MODE',
 'BASEMENTAREA_MODE',
 'YEARS_BEGINEXPLUATATION_MODE',
 'YEARS_BUILD_MODE',
 'COMMONAREA_MODE',
 'ELEVATORS_MODE',
 'ENTRANCES_MODE',
 'FLOORSMAX_MODE',
 'FLOORSMIN_MODE',
 'LANDAREA_MODE',
 'LIVINGAPARTMENTS_MODE',
 'LIVINGAREA_MODE',
 'NONLIVINGAPARTMENTS_MODE',
 'NONLIVINGAREA_MODE',
 'APARTMENTS_MEDI',
 'BASEMENTAREA_MEDI',
 'YEARS_BEGINEXPLUATATION_MEDI',
 'YEARS_BUILD_MEDI',
 'COMMONAREA_MEDI',
 'ELEVATORS_MEDI',
 'ENTRANCES_MEDI',
 'FLOORSMAX_MEDI',
 'FLOORSMIN_MEDI',
 'LANDAREA_MEDI',
 'LIVINGAPARTMENTS_MEDI',
 'LIVINGAREA_MEDI',
 'NONLIVINGAPARTMENTS_MEDI',
 'NONLIVINGAREA_MEDI',
 'TOTALAREA_MODE',
 'OBS_60_CNT_SOCIAL_CIRCLE',
 'CODE_GENDER_M',
 'NAME_INCOME_TYPE_Pensioner',
 'ORGANIZATION_TYPE_XNA',
 'EMERGENCYSTATE_MODE_No']

In [119]:
train = train.drop(columns = to_drop)
test = test.drop(columns = to_drop)

print('Training shape: ', train.shape)
print('Testing shape: ', test.shape)

Training shape:  (418521, 201)
Testing shape:  (48744, 200)


In [120]:
train

,SK_ID_CURR,NAME_CONTRACT_TYPE,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,REGION_POPULATION_RELATIVE,DAYS_BIRTH,...,HOUSETYPE_MODE_terraced house,WALLSMATERIAL_MODE_Block,WALLSMATERIAL_MODE_Mixed,WALLSMATERIAL_MODE_Monolithic,WALLSMATERIAL_MODE_Others,WALLSMATERIAL_MODE_Panel,"WALLSMATERIAL_MODE_Stone, brick",WALLSMATERIAL_MODE_Wooden,EMERGENCYSTATE_MODE_Yes,TARGET
0,100002,0,0,1,0,202500.0,406597.5,24700.5,0.018801,-9461,...,0,0,0,0,0,0,1,0,0,1
1,100002,0,0,1,0,202500.0,406597.5,24700.5,0.018801,-9461,...,0,0,0,0,0,0,1,0,0,1
2,100003,0,0,0,0,270000.0,1293502.5,35698.5,0.003541,-16765,...,0,1,0,0,0,0,0,0,0,0
3,100003,0,0,0,0,270000.0,1293502.5,35698.5,0.003541,-16765,...,0,1,0,0,0,0,0,0,0,0
4,100004,1,1,1,0,67500.0,135000.0,6750.0,0.010032,-19046,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
418516,456251,0,0,0,0,157500.0,254700.0,27558.0,0.032561,-9327,...,0,0,0,0,0,0,1,0,0,0
418517,456252,0,0,1,0,72000.0,269550.0,12001.5,0.025164,-20775,...,0,0,0,0,0,0,1,0,0,0
418518,456253,0,0,1,0,153000.0,677664.0,29979.0,0.005002,-14966,...,0,0,0,0,0,1,0,0,0,0
418519,456254,0,0,1,0,171000.0,370107.0,20205.0,0.005313,-11961,...,0,0,0,0,0,0,1,0,0,1


In [121]:
test

,SK_ID_CURR,NAME_CONTRACT_TYPE,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,REGION_POPULATION_RELATIVE,DAYS_BIRTH,...,HOUSETYPE_MODE_specific housing,HOUSETYPE_MODE_terraced house,WALLSMATERIAL_MODE_Block,WALLSMATERIAL_MODE_Mixed,WALLSMATERIAL_MODE_Monolithic,WALLSMATERIAL_MODE_Others,WALLSMATERIAL_MODE_Panel,"WALLSMATERIAL_MODE_Stone, brick",WALLSMATERIAL_MODE_Wooden,EMERGENCYSTATE_MODE_Yes
0,100001,0,0,1,0,135000.0,568800.0,20560.5,0.018850,-19241,...,0,0,0,0,0,0,0,1,0,0
1,100005,0,0,1,0,99000.0,222768.0,17370.0,0.035792,-18064,...,0,0,0,0,0,0,0,0,0,0
2,100013,0,1,1,0,202500.0,663264.0,69777.0,0.019101,-20038,...,0,0,0,0,0,0,0,0,0,0
3,100028,0,0,1,2,315000.0,1575000.0,49018.5,0.026392,-13976,...,0,0,0,0,0,0,1,0,0,0
4,100038,0,1,0,1,180000.0,625500.0,32067.0,0.010032,-13040,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48739,456221,0,0,1,0,121500.0,412560.0,17473.5,0.002042,-19970,...,0,0,0,0,0,0,0,0,0,0
48740,456222,0,0,0,2,157500.0,622413.0,31909.5,0.035792,-11186,...,0,0,0,0,0,0,0,0,0,0
48741,456223,0,1,1,1,202500.0,315000.0,33205.5,0.026392,-15922,...,0,0,0,0,0,0,0,1,0,0
48742,456224,0,0,0,0,225000.0,450000.0,25128.0,0.018850,-13968,...,0,0,0,0,0,0,1,0,0,0


In [122]:
#train.describe()


In [123]:
correlations = train.corr()['TARGET'].sort_values()
correlations

EXT_SOURCE_3                           -0.179433
EXT_SOURCE_2                           -0.159119
EXT_SOURCE_1                           -0.154810
NAME_EDUCATION_TYPE_Higher education   -0.056592
CODE_GENDER_F                          -0.053823
                                          ...   
DAYS_LAST_PHONE_CHANGE                  0.054436
NAME_INCOME_TYPE_Working                0.056670
REGION_RATING_CLIENT                    0.058562
DAYS_BIRTH                              0.078222
TARGET                                  1.000000
Name: TARGET, Length: 201, dtype: float64

In [124]:
# Display correlations
print('Most Positive Correlations:\n', correlations.tail(15))
print('\nMost Negative Correlations:\n', correlations.head(15))

Most Positive Correlations:
 DEF_30_CNT_SOCIAL_CIRCLE                             0.032125
LIVE_CITY_NOT_WORK_CITY                              0.032561
OWN_CAR_AGE                                          0.037526
DAYS_REGISTRATION                                    0.040551
REG_CITY_NOT_LIVE_CITY                               0.043193
OCCUPATION_TYPE_Laborers                             0.043515
FLAG_DOCUMENT_3                                      0.044129
NAME_EDUCATION_TYPE_Secondary / secondary special    0.049709
DAYS_ID_PUBLISH                                      0.050109
REG_CITY_NOT_WORK_CITY                               0.050423
DAYS_LAST_PHONE_CHANGE                               0.054436
NAME_INCOME_TYPE_Working                             0.056670
REGION_RATING_CLIENT                                 0.058562
DAYS_BIRTH                                           0.078222
TARGET                                               1.000000
Name: TARGET, dtype: float64

Most Negati

In [125]:
# Train missing values (in percent)
train_missing = (train.isnull().sum() / len(train)).sort_values(ascending = False)
train_missing.head()

COMMONAREA_AVG             0.697542
NONLIVINGAPARTMENTS_AVG    0.693184
FLOORSMIN_AVG              0.677316
YEARS_BUILD_AVG            0.663799
OWN_CAR_AGE                0.658041
dtype: float64

In [126]:
# Test missing values (in percent)
test_missing = (test.isnull().sum() / len(test)).sort_values(ascending = False)
test_missing.head()

COMMONAREA_AVG             0.687161
NONLIVINGAPARTMENTS_AVG    0.684125
FLOORSMIN_AVG              0.666051
OWN_CAR_AGE                0.662892
YEARS_BUILD_AVG            0.652757
dtype: float64

In [127]:
# Identify missing values above threshold
train_missing = train_missing.index[train_missing > 0.75]
test_missing = test_missing.index[test_missing > 0.75]


In [128]:
all_missing = list(set(set(train_missing) | set(test_missing)))
print('There are %d columns with more than 75%% missing values' % len(all_missing))


There are 0 columns with more than 75% missing values


In [129]:
# Need to save the labels because aligning will remove this column
train_labels = train["TARGET"]
train_ids = train['SK_ID_CURR']
test_ids = test['SK_ID_CURR']

In [ ]:
#train = pd.get_dummies(train.drop(columns = all_missing))
#test = pd.get_dummies(test.drop(columns = all_missing))

In [130]:
train, test = train.align(test, join = 'inner', axis = 1)

In [131]:
print('Training set full shape: ', train.shape)
print('Testing set full shape: ' , test.shape)

Training set full shape:  (418521, 200)
Testing set full shape:  (48744, 200)


In [132]:
train = train.drop(columns = ['SK_ID_CURR'])
test = test.drop(columns = ['SK_ID_CURR'])

In [133]:
# Initialize an empty array to hold feature importances
feature_importances = np.zeros(train.shape[1])

In [135]:
# Create the model with several hyperparameters
#model = lgb.LGBMClassifier(objective='binary', boosting_type = 'goss', n_estimators = 10000, class_weight = 'balanced')

In [ ]:
# Fit the model twice to avoid overfitting
#for i in range(2):

In [ ]:
# Split into training and validation set
#train_features, valid_features, train_y, valid_y = train_test_split(train, train_labels, test_size = 0.25, random_state = i)


In [ ]:
# Train using early stopping
#model.fit(train_features, train_y, early_stopping_rounds=100, eval_set = [(valid_features, valid_y)], 
#eval_metric = 'auc', verbose = 200)


In [ ]:
# Record the feature importances
#    feature_importances += model.feature_importances

In [ ]:
# Make sure to average feature importances! 
#feature_importances = feature_importances / 2
#feature_importances = pd.DataFrame({'feature': list(train.columns), 'importance': feature_importances}).sort_values('importance', ascending = False)


In [ ]:
#feature_importances.head()

In [ ]:
# Find the features with zero importance
#zero_features = list(feature_importances[feature_importances['importance'] == 0.0]['feature'])
#print('There are %d features with 0.0 importance' % len(zero_features))
#feature_importances.tail()

In [136]:
def plot_feature_importances(df, threshold = 0.9):
    """
    Plots 15 most important features and the cumulative importance of features.
    Prints the number of features needed to reach threshold cumulative importance.
    
    Parameters
    --------
    df : dataframe
        Dataframe of feature importances. Columns must be feature and importance
    threshold : float, default = 0.9
        Threshold for prining information about cumulative importances
        
    Return
    --------
    df : dataframe
        Dataframe ordered by feature importances with a normalized column (sums to 1)
        and a cumulative importance column
    
    """
    
    plt.rcParams['font.size'] = 18
    
    # Sort features according to importance
    df = df.sort_values('importance', ascending = False).reset_index()
    
    # Normalize the feature importances to add up to one
    df['importance_normalized'] = df['importance'] / df['importance'].sum()
    df['cumulative_importance'] = np.cumsum(df['importance_normalized'])

    # Make a horizontal bar chart of feature importances
    plt.figure(figsize = (10, 6))
    ax = plt.subplot()
    
    # Need to reverse the index to plot most important on top
    ax.barh(list(reversed(list(df.index[:15]))), 
            df['importance_normalized'].head(15), 
            align = 'center', edgecolor = 'k')
    
    # Set the yticks and labels
    ax.set_yticks(list(reversed(list(df.index[:15]))))
    ax.set_yticklabels(df['feature'].head(15))
    
    # Plot labeling
    plt.xlabel('Normalized Importance'); plt.title('Feature Importances')
    plt.show()
    
    # Cumulative importance plot
    plt.figure(figsize = (8, 6))
    plt.plot(list(range(len(df))), df['cumulative_importance'], 'r-')
    plt.xlabel('Number of Features'); plt.ylabel('Cumulative Importance'); 
    plt.title('Cumulative Feature Importance');
    plt.show();
    
    importance_index = np.min(np.where(df['cumulative_importance'] > threshold))
    print('%d features required for %0.2f of cumulative importance' % (importance_index + 1, threshold))
    
    return df

In [137]:
train = train.drop(columns = zero_features)
test = test.drop(columns = zero_features)

print('Training shape: ', train.shape)
print('Testing shape: ', test.shape)

NameError: name 'zero_features' is not defined

In [138]:
#Aplica de nuevo el modelo
def identify_zero_importance_features(train, train_labels, iterations = 2):
    """
    Identify zero importance features in a training dataset based on the 
    feature importances from a gradient boosting model. 
    
    Parameters
    --------
    train : dataframe
        Training features
        
    train_labels : np.array
        Labels for training data
        
    iterations : integer, default = 2
        Number of cross validation splits to use for determining feature importances
    """
    
    # Initialize an empty array to hold feature importances
    feature_importances = np.zeros(train.shape[1])

    # Create the model with several hyperparameters
    model = lgb.LGBMClassifier(objective='binary', boosting_type = 'goss', n_estimators = 10000, class_weight = 'balanced')
    
    # Fit the model multiple times to avoid overfitting
    for i in range(iterations):

        # Split into training and validation set
        train_features, valid_features, train_y, valid_y = train_test_split(train, train_labels, test_size = 0.25, random_state = i)

        # Train using early stopping
        model.fit(train_features, train_y, early_stopping_rounds=100, eval_set = [(valid_features, valid_y)], 
                  eval_metric = 'auc', verbose = 200)

        # Record the feature importances
        feature_importances += model.feature_importances_ / iterations
    
    feature_importances = pd.DataFrame({'feature': list(train.columns), 'importance': feature_importances}).sort_values('importance', ascending = False)
    
    # Find the features with zero importance
    zero_features = list(feature_importances[feature_importances['importance'] == 0.0]['feature'])
    print('\nThere are %d features with 0.0 importance' % len(zero_features))
    
    return zero_features, feature_importances

second_round_zero_features, feature_importances = identify_zero_importance_features(train, train_labels)


NameError: name 'lgb' is not defined

In [139]:
#from sklearn.preprocessing import MinMaxScaler, Imputer

# Drop the target from the training data
if 'TARGET' in train:
    train = train.drop(columns = ['TARGET'])
else:
    train = train.copy()
    
# Feature names
features = list(train.columns)

# Copy of the testing data
test = test.copy()

# Median imputation of missing values
imputer = SimpleImputer(strategy = 'median')

# Scale each feature to 0-1
scaler = MinMaxScaler(feature_range = (0, 1))

# Fit on the training data
imputer.fit(train)

# Transform both training and testing data
train = imputer.transform(train)
test = imputer.transform(test)

# Repeat with the scaler
scaler.fit(train)
train = scaler.transform(train)
test = scaler.transform(test)

print('Training data shape: ', train.shape)
print('Testing data shape: ', test.shape)

Training data shape:  (418521, 199)
Testing data shape:  (48744, 199)


In [140]:
from sklearn.linear_model import LogisticRegression

# Make the model with the specified regularization parameter
log_reg = LogisticRegression(C = 0.0001)

# Train on the training data
log_reg.fit(train, train_labels)

LogisticRegression(C=0.0001, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [141]:
# Make predictions
# Make sure to select the second column only
log_reg_pred = log_reg.predict_proba(test)[:, 1]

In [142]:
a = np.array(log_reg_pred)
print(a)
a[0]

[0.0635077  0.1166396  0.06089404 ... 0.06665915 0.06956687 0.08994502]


0.06350769863417108

In [143]:
# Submission dataframe
# Make the submission dataframe
submit = pd.DataFrame({'SK_ID_CURR': test, 'TARGET': log_reg_pred})

#submit = test['SK_ID_CURR']
#submit['TARGET'] = log_reg_pred

submit.head()

Exception: Data must be 1-dimensional

In [144]:
#from sklearn.ensemble import RandomForestClassifier

# Make the random forest classifier
random_forest = RandomForestClassifier(n_estimators = 100, random_state = 50, verbose = 1, n_jobs = -1)

In [145]:
random_forest.fit(train, train_labels)

# Extract feature importances
feature_importance_values = random_forest.feature_importances_
feature_importances = pd.DataFrame({'feature': features, 'importance': feature_importance_values})

# Make predictions on the test data
predictions = random_forest.predict_proba(test)[:, 1]

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   31.0s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:  1.3min finished
[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.1s
[Parallel(n_jobs=8)]: Done 100 out of 100 | elapsed:    0.6s finished


In [146]:
#print(predictions)
b = np.array(predictions)
print(b)
b[0]

[0.09 0.1  0.04 ... 0.04 0.04 0.16]


0.09

In [149]:
# Make a submission dataframe
submit = pd.DataFrame({test.SK_ID_CURR, test.TARGET, predictions})
#submit = test[['SK_ID_CURR']]
#submit['TARGET'] = predictions
submit

AttributeError: 'numpy.ndarray' object has no attribute 'SK_ID_CURR'

In [162]:
#from sklearn.model_selection import KFold
#from sklearn.metrics import roc_auc_score
#import lightgbm as lgb
#import gc

def model(features, test_features, encoding = 'ohe', n_folds = 5):
    
    """Train and test a light gradient boosting model using
    cross validation. 
    
    Parameters
    --------
        features (pd.DataFrame): 
            dataframe of training features to use 
            for training a model. Must include the TARGET column.
        test_features (pd.DataFrame): 
            dataframe of testing features to use
            for making predictions with the model. 
        encoding (str, default = 'ohe'): 
            method for encoding categorical variables. Either 'ohe' for one-hot encoding or 'le' for integer label encoding
            n_folds (int, default = 5): number of folds to use for cross validation
        
    Return
    --------
        submission (pd.DataFrame): 
            dataframe with `SK_ID_CURR` and `TARGET` probabilities
            predicted by the model.
        feature_importances (pd.DataFrame): 
            dataframe with the feature importances from the model.
        valid_metrics (pd.DataFrame): 
            dataframe with training and validation metrics (ROC AUC) for each fold and overall.
        
    """
    
    # Extract the ids
    train_ids = features['SK_ID_CURR']
    test_ids = test_features['SK_ID_CURR']
    
    # Extract the labels for training
    labels = features['TARGET']
    
    # Remove the ids and target
    features = features.drop(columns = ['SK_ID_CURR', 'TARGET'])
    test_features = test_features.drop(columns = ['SK_ID_CURR'])
    
    
    # One Hot Encoding
    if encoding == 'ohe':
        features = pd.get_dummies(features)
        test_features = pd.get_dummies(test_features)
        
        # Align the dataframes by the columns
        features, test_features = features.align(test_features, join = 'inner', axis = 1)
        
        # No categorical indices to record
        cat_indices = 'auto'
    
    # Integer label encoding
    elif encoding == 'le':
        
        # Create a label encoder
        label_encoder = LabelEncoder()
        
        # List for storing categorical indices
        cat_indices = []
        
        # Iterate through each column
        for i, col in enumerate(features):
            if features[col].dtype == 'object':
                # Map the categorical features to integers
                features[col] = label_encoder.fit_transform(np.array(features[col].astype(str)).reshape((-1,)))
                test_features[col] = label_encoder.transform(np.array(test_features[col].astype(str)).reshape((-1,)))

                # Record the categorical indices
                cat_indices.append(i)
    
    # Catch error if label encoding scheme is not valid
    else:
        raise ValueError("Encoding must be either 'ohe' or 'le'")
        
    print('Training Data Shape: ', features.shape)
    print('Testing Data Shape: ', test_features.shape)
    
    # Extract feature names
    feature_names = list(features.columns)
    
    # Convert to np arrays
    features = np.array(features)
    test_features = np.array(test_features)
    
    # Create the kfold object
    k_fold = KFold(n_splits = n_folds, shuffle = True, random_state = 50)
    
    # Empty array for feature importances
    feature_importance_values = np.zeros(len(feature_names))
    
    # Empty array for test predictions
    test_predictions = np.zeros(test_features.shape[0])
    
    # Empty array for out of fold validation predictions
    out_of_fold = np.zeros(features.shape[0])
    
    # Lists for recording validation and training scores
    valid_scores = []
    train_scores = []
    
    # Iterate through each fold
    for train_indices, valid_indices in k_fold.split(features):
        
        # Training data for the fold
        train_features, train_labels = features[train_indices], labels[train_indices]
        # Validation data for the fold
        valid_features, valid_labels = features[valid_indices], labels[valid_indices]
        
        # Create the model
        model = lgb.LGBMClassifier(n_estimators=10000, objective = 'binary', 
                                   class_weight = 'balanced', learning_rate = 0.05, 
                                   reg_alpha = 0.1, reg_lambda = 0.1, 
                                   subsample = 0.8, n_jobs = -1, random_state = 50)
        
        # Train the model
        model.fit(train_features, train_labels, eval_metric = 'auc',
                  eval_set = [(valid_features, valid_labels), (train_features, train_labels)],
                  eval_names = ['valid', 'train'], categorical_feature = cat_indices,
                  early_stopping_rounds = 100, verbose = 200)
        
        # Record the best iteration
        best_iteration = model.best_iteration_
        
        # Record the feature importances
        feature_importance_values += model.feature_importances_ / k_fold.n_splits
        
        # Make predictions
        test_predictions += model.predict_proba(test_features, num_iteration = best_iteration)[:, 1] / k_fold.n_splits
        
        # Record the out of fold predictions
        out_of_fold[valid_indices] = model.predict_proba(valid_features, num_iteration = best_iteration)[:, 1]
        
        # Record the best score
        valid_score = model.best_score_['valid']['auc']
        train_score = model.best_score_['train']['auc']
        
        valid_scores.append(valid_score)
        train_scores.append(train_score)
        
        # Clean up memory
        gc.enable()
        del model, train_features, valid_features
        gc.collect()
        
    # Make the submission dataframe
    submission = pd.DataFrame({'SK_ID_CURR': test_ids, 'TARGET': test_predictions})
    
    # Make the feature importance dataframe
    feature_importances = pd.DataFrame({'feature': feature_names, 'importance': feature_importance_values})
    
    # Overall validation score
    valid_auc = roc_auc_score(labels, out_of_fold)
    
    # Add the overall scores to the metrics
    valid_scores.append(valid_auc)
    train_scores.append(np.mean(train_scores))
    
    # Needed for creating dataframe of validation scores
    fold_names = list(range(n_folds))
    fold_names.append('overall')
    
    # Dataframe of validation scores
    metrics = pd.DataFrame({'fold': fold_names,
                            'train': train_scores,
                            'valid': valid_scores}) 
    
    return submission, feature_importances, metrics

In [164]:
#submission, fi, metrics = model(train, test)
#print('Baseline metrics')
submission, fi, metrics = model(features, test_features)
print(metrics)

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [165]:
def model(features, test_features, encoding = 'ohe', n_folds = 5):
    
    """Train and test a light gradient boosting model using
    cross validation. 
    
    Parameters
    --------
        features (pd.DataFrame): 
            dataframe of training features to use 
            for training a model. Must include the TARGET column.
        test_features (pd.DataFrame): 
            dataframe of testing features to use
            for making predictions with the model. 
        encoding (str, default = 'ohe'): 
            method for encoding categorical variables. Either 'ohe' for one-hot encoding or 'le' for integer label encoding
            n_folds (int, default = 5): number of folds to use for cross validation
        
    Return
    --------
        submission (pd.DataFrame): 
            dataframe with `SK_ID_CURR` and `TARGET` probabilities
            predicted by the model.
        feature_importances (pd.DataFrame): 
            dataframe with the feature importances from the model.
        valid_metrics (pd.DataFrame): 
            dataframe with training and validation metrics (ROC AUC) for each fold and overall.
        
    """
    
    # Extract the ids
    train_ids = features['SK_ID_CURR']
    test_ids = test_features['SK_ID_CURR']
    
    # Extract the labels for training
    labels = features['TARGET']
    
    # Remove the ids and target
    features = features.drop(columns = ['SK_ID_CURR', 'TARGET'])
    test_features = test_features.drop(columns = ['SK_ID_CURR'])
    
    
    # One Hot Encoding
    if encoding == 'ohe':
        features = pd.get_dummies(features)
        test_features = pd.get_dummies(test_features)
        
        # Align the dataframes by the columns
        features, test_features = features.align(test_features, join = 'inner', axis = 1)
        
        # No categorical indices to record
        cat_indices = 'auto'
    
    # Integer label encoding
    elif encoding == 'le':
        
        # Create a label encoder
        label_encoder = LabelEncoder()
        
        # List for storing categorical indices
        cat_indices = []
        
        # Iterate through each column
        for i, col in enumerate(features):
            if features[col].dtype == 'object':
                # Map the categorical features to integers
                features[col] = label_encoder.fit_transform(np.array(features[col].astype(str)).reshape((-1,)))
                test_features[col] = label_encoder.transform(np.array(test_features[col].astype(str)).reshape((-1,)))

                # Record the categorical indices
                cat_indices.append(i)
    
    # Catch error if label encoding scheme is not valid
    else:
        raise ValueError("Encoding must be either 'ohe' or 'le'")
        
    print('Training Data Shape: ', features.shape)
    print('Testing Data Shape: ', test_features.shape)
    
    # Extract feature names
    feature_names = list(features.columns)
    
    # Convert to np arrays
    features = np.array(features)
    test_features = np.array(test_features)
    
    # Create the kfold object
    k_fold = KFold(n_splits = n_folds, shuffle = False, random_state = 50)
    
    # Empty array for feature importances
    feature_importance_values = np.zeros(len(feature_names))
    
    # Empty array for test predictions
    test_predictions = np.zeros(test_features.shape[0])
    
    # Empty array for out of fold validation predictions
    out_of_fold = np.zeros(features.shape[0])
    
    # Lists for recording validation and training scores
    valid_scores = []
    train_scores = []
    
    # Iterate through each fold
    for train_indices, valid_indices in k_fold.split(features):
        
        # Training data for the fold
        train_features, train_labels = features[train_indices], labels[train_indices]
        # Validation data for the fold
        valid_features, valid_labels = features[valid_indices], labels[valid_indices]
        
        # Create the model
        model = lgb.LGBMClassifier(n_estimators=10000, objective = 'binary', 
                                   class_weight = 'balanced', learning_rate = 0.05, 
                                   reg_alpha = 0.1, reg_lambda = 0.1, 
                                   subsample = 0.8, n_jobs = -1, random_state = 50)
        
        # Train the model
        model.fit(train_features, train_labels, eval_metric = 'auc',
                  eval_set = [(valid_features, valid_labels), (train_features, train_labels)],
                  eval_names = ['valid', 'train'], categorical_feature = cat_indices,
                  early_stopping_rounds = 100, verbose = 200)
        
        # Record the best iteration
        best_iteration = model.best_iteration_
        
        # Record the feature importances
        feature_importance_values += model.feature_importances_ / k_fold.n_splits
        
        # Make predictions
        test_predictions += model.predict_proba(test_features, num_iteration = best_iteration)[:, 1] / k_fold.n_splits
        
        # Record the out of fold predictions
        out_of_fold[valid_indices] = model.predict_proba(valid_features, num_iteration = best_iteration)[:, 1]
        
        # Record the best score
        valid_score = model.best_score_['valid']['auc']
        train_score = model.best_score_['train']['auc']
        
        valid_scores.append(valid_score)
        train_scores.append(train_score)
        
        # Clean up memory
        gc.enable()
        del model, train_features, valid_features
        gc.collect()
        
    # Make the submission dataframe
    submission = pd.DataFrame({'SK_ID_CURR': test_ids, 'TARGET': test_predictions})
    
    # Make the feature importance dataframe
    feature_importances = pd.DataFrame({'feature': feature_names, 'importance': feature_importance_values})
    
    # Overall validation score
    valid_auc = roc_auc_score(labels, out_of_fold)
    
    # Add the overall scores to the metrics
    valid_scores.append(valid_auc)
    train_scores.append(np.mean(train_scores))
    
    # Needed for creating dataframe of validation scores
    fold_names = list(range(n_folds))
    fold_names.append('overall')
    
    # Dataframe of validation scores
    metrics = pd.DataFrame({'fold': fold_names,
                            'train': train_scores,
                            'valid': valid_scores}) 
    
    return submission, feature_importances, metrics

In [166]:
submission, fi, metrics = model(features, test_features)

NameError: name 'test_features' is not defined

In [168]:
#from sklearn.decomposition import PCA
#from sklearn.preprocessing import Imputer
#from sklearn.pipeline import Pipeline

# Make sure to drop the ids and target
train = train.drop(columns = ['SK_ID_CURR', 'TARGET'])
test = test.drop(columns = ['SK_ID_CURR'])

# Make a pipeline with imputation and pca
pipeline = Pipeline(steps = [('imputer', SimpleImputer(strategy = 'median')),
             ('pca', PCA())])

# Fit and transform on the training data
train_pca = pipeline.fit_transform(train)

# transform the testing data
test_pca = pipeline.transform(test)

AttributeError: 'numpy.ndarray' object has no attribute 'drop'

In [169]:
# Extract the pca object
pca = pipeline.named_steps['pca']

# Plot the cumulative variance explained

plt.figure(figsize = (10, 8))
plt.plot(list(range(train.shape[1])), np.cumsum(pca.explained_variance_ratio_), 'r-')
plt.xlabel('Number of PC'); plt.ylabel('Cumulative Variance Explained');
plt.title('Cumulative Variance Explained with PCA');

NameError: name 'pipeline' is not defined